# HPC intro

## atools

There are many situations in which you want to run an application for (potentially many) different input parameters.  These parameters can be command line options you run your application with, or file names you provide and so on.

Of course, you could submit a job for each of the instances of your problem, but that would result in many jobs.  Moreover, quite some bookkeeping would be required if some instances fail, while others succeed.  You typically don't have a convenient way to get an overview of which instances failed, and hence have to be redone.

Alternatively, you could simply do all these instances looping over all the parameters.  This would result in potentially prohibitively long run times, and, more importantly, you would not be exploiting a supercomputers main feature: executing work in parallel.

atools has been designed to make it easy for you to run many instances of a problem in parallel, and it takes care of the bookkeeping for you as well.  An instance of the problem that you want to compute is called a *work item* in the context of atools.

### Job script

The first step is to make a few modifications to your job script.  By way of example, use a script that simply calculates and displays the product of two numbers that you also used in the [tutorial on jobs](020_jobs.ipynb).

```bash
#!/usr/bin/env -S bash -l
#SBATCH --account=lp_tutorial
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=1g
#SBATCH --time=00:05:00

# actual computation, a bit boring
for i in $(seq 1 10)
do
    for j in $(seq 1 10)
    do
        echo $(( $i * $j ))
    done
done
```

In this job script, you do all computations sequentially, but to speed things up, you would like to do them in parallel as independent jobs.  So you can rewrite the job script such that it only does a single multiplication.

```bash
#!/usr/bin/env -S bash -l
#SBATCH --account=lp_multiscale_physics
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=1g
#SBATCH --time=00:05:00

# actual computation, a bit boring
echo $(( $i * $j ))
```

The job script has been adapted to compute a single work item.

This is where atools comes in.  You can make a few more modifications to this job script to use it.  The values of `i` and `j` will be read from a Comma Separated Value file (CSV file).

The first line of this file lists the names of the variables, each line after that the values that correspond to the work items.  So for this example, that would look as follows.  

```
i,j
1,1
1,2
1,3
...
10,8
10,9
10,10
```

You don't have to type all that, there is a data file `data.csv` available that you can copy.  You can find it in the `021_artefacts` directory.

As it is, this script would fail since at this point the variables `i` and `j` are not defined.  You have to make sure that atools can do its magic.  For that purpose, you have to make a few more modifications to the job script.

1. Load the `atools` module.
2. Log the start of the work item.
3. Make sure that the variables used in the script are initialized.
4. Log the end of the work item.

```bash
#!/usr/bin/env -S bash -l
#SBATCH --account=lp_tutorial
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=1g
#SBATCH --time=00:05:00

# make sure the module system starts from a clean slate and load the atools module
module purge
module load atools

# log the start of the work item
alog  --state start

# initialize the variables
source <(aenv --data data.csv)

# actual computation, a bit boring
echo $(( $i * $j ))

# log the end of the work item
alog  --state end  --exit $?
```

Now your job script is fully adapted to use atools features.  It is available in the `021_artefacts` directory as `jobscript_parallel.slurm`.   Don't forget to change the credit account name to the one you have access to.

### Job submission

You can submit an atools job almost the same way as an ordinary job, except that you need to specify the `--array` option for `sbatch`.  If you know the number of work items, 100 in the `data.csv` file you are using, you can simply use `--array=1-100`.  Otherwise, atools can help you determine it easily.

First, load the atools module.

In [1]:
module load atools

Next, submit the job as follows.

In [2]:
sbatch  --cluster=wice  --array=$(arange --data data.csv)  jobscript_parallel.slurm

Submitted batch job 60666714 on cluster wice


When you check the queue, you will notice that
* when your job is not running yet, the job ID is somewhat unusual, `_[1-100]` is appended to it;
* when your job has started to run, you will see many entries where a single number was appended to the job ID; these are the indivitual work items.

In [3]:
squeue --cluster=wice

CLUSTER: wice
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
  60666714_[1-100]     batch jobscrip vsc30032 PD       0:00      1 (Priority)
          60666711 interacti sys/dash vsc30032  R       2:53      1 k28i14


In [4]:
squeue --cluster=wice

CLUSTER: wice
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
  60666714_[1-100]     batch jobscrip vsc30032 PD       0:00      1 (Priority)
          60666711 interacti sys/dash vsc30032  R       3:02      1 k28i14


In [5]:
squeue --cluster=wice

CLUSTER: wice
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
 60666714_[21-100]     batch jobscrip vsc30032 PD       0:00      1 (Priority)
        60666714_3     batch jobscrip vsc30032  R       0:19      1 s28c11n2
        60666714_4     batch jobscrip vsc30032  R       0:19      1 s28c11n2
        60666714_5     batch jobscrip vsc30032  R       0:19      1 s28c11n2
        60666714_6     batch jobscrip vsc30032  R       0:19      1 s28c11n2
        60666714_8     batch jobscrip vsc30032  R       0:19      1 s28c11n2
        60666714_9     batch jobscrip vsc30032  R       0:19      1 s28c11n2
       60666714_10     batch jobscrip vsc30032  R       0:19      1 s28c11n2
       60666714_11     batch jobscrip vsc30032  R       0:19      1 s28c11n2
       60666714_12     batch jobscrip vsc30032  R       0:19      1 s28c11n2
       60666714_13     batch jobscrip vsc30032  R       0:19      1 s28c11n2
       60666714_14     batch jobscrip vsc30032  R   

In [6]:
squeue --cluster=wice

CLUSTER: wice
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
 60666714_[21-100]     batch jobscrip vsc30032 PD       0:00      1 (Priority)
       60666714_10     batch jobscrip vsc30032  R       0:39      1 s28c11n2
          60666711 interacti sys/dash vsc30032  R       3:46      1 k28i14


In [9]:
squeue --cluster=wice

CLUSTER: wice
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          60666711 interacti sys/dash vsc30032  R      11:19      1 k28i14


When the job finishes, you will notice a lot of files, each containing the output of a single work item.  You should have as many as there are work items.

In [10]:
ls slurm-*

slurm-60666714_100.out	slurm-60666714_40.out  slurm-60666714_71.out
slurm-60666714_10.out	slurm-60666714_41.out  slurm-60666714_72.out
slurm-60666714_11.out	slurm-60666714_42.out  slurm-60666714_73.out
slurm-60666714_12.out	slurm-60666714_43.out  slurm-60666714_74.out
slurm-60666714_13.out	slurm-60666714_44.out  slurm-60666714_75.out
slurm-60666714_14.out	slurm-60666714_45.out  slurm-60666714_76.out
slurm-60666714_15.out	slurm-60666714_46.out  slurm-60666714_77.out
slurm-60666714_16.out	slurm-60666714_47.out  slurm-60666714_78.out
slurm-60666714_17.out	slurm-60666714_48.out  slurm-60666714_79.out
slurm-60666714_18.out	slurm-60666714_49.out  slurm-60666714_7.out
slurm-60666714_19.out	slurm-60666714_4.out   slurm-60666714_80.out
slurm-60666714_1.out	slurm-60666714_50.out  slurm-60666714_81.out
slurm-60666714_20.out	slurm-60666714_51.out  slurm-60666714_82.out
slurm-60666714_21.out	slurm-60666714_52.out  slurm-60666714_83.out
slurm-60666714_22.out	slurm-60666714_53.out  slurm-60666714_84.

In [11]:
ls slurm-*.out | wc -l

100


In [12]:
cat slurm-*_100.out

SLURM_JOB_ID: 60666714
SLURM_JOB_USER: vsc30032
SLURM_JOB_ACCOUNT: lpt2_sysadmin
SLURM_JOB_NAME: jobscript_parallel.slurm
SLURM_CLUSTER_NAME: wice
SLURM_JOB_PARTITION: batch
SLURM_ARRAY_JOB_ID: 60666714
SLURM_ARRAY_TASK_ID: 100
SLURM_NNODES: 1
SLURM_NODELIST: m28c20n2
SLURM_JOB_CPUS_PER_NODE: 1
Date: Thu Aug 31 16:57:23 CEST 2023
Walltime: 00-00:05:00
100


Indeed, work item 100 would be the multiplication of the values 10 and 10.  It is the last line in `data.csv` and hence your last work item.

Since this is just a tutorial job, you probably would like to remove these files.

In [13]:
rm slurm-*.out

### Log file

You will remember that you enabled logging using the `alog` commands in `jobscript_parallel.slurm`.  This has resulted in a file that contains information about the work items' execution:

  1. its number
  1. when they started,
  1. the name of the compute node the work item was executed on,
  1. when they completed, and
  1. the exit status, if there was a failure.
  
The name of this job script is the name of your job, with `.log` appended, followed by the job ID.  The command below shows you the first 20 lines.

In [14]:
head -20 jobscript_parallel.slurm.log*

18 started by s28c11n2 at 2023-08-31 16:52:23
1 started by m28c20n2 at 2023-08-31 16:52:23
2 started by s28c11n2 at 2023-08-31 16:52:25
17 started by s28c11n2 at 2023-08-31 16:52:26
7 started by s28c11n2 at 2023-08-31 16:52:26
20 started by s28c11n2 at 2023-08-31 16:52:28
4 started by s28c11n2 at 2023-08-31 16:52:28
9 started by s28c11n2 at 2023-08-31 16:52:29
19 started by s28c11n2 at 2023-08-31 16:52:29
16 started by s28c11n2 at 2023-08-31 16:52:31
8 started by s28c11n2 at 2023-08-31 16:52:31
6 started by s28c11n2 at 2023-08-31 16:52:32
18 completed by s28c11n2 at 2023-08-31 16:52:33
1 completed by m28c20n2 at 2023-08-31 16:52:33
11 started by s28c11n2 at 2023-08-31 16:52:35
2 completed by s28c11n2 at 2023-08-31 16:52:36
17 completed by s28c11n2 at 2023-08-31 16:52:36
13 started by s28c11n2 at 2023-08-31 16:52:36
7 completed by s28c11n2 at 2023-08-31 16:52:36
20 completed by s28c11n2 at 2023-08-31 16:52:38


You could of course eyeball this file to determine whether all work items completed succesfully, but atools has a command to simplify that considerably.

In [15]:
arange  --data data.csv  --log jobscript_parallel.slurm.log*  --summary

Summary:
  items completed: 100
  items failed: 0
  items to do: 0


As you can see, 100 items were completed, none failed, and none left to do.